<a href="https://colab.research.google.com/github/JKH-ML/python/blob/main/11_K_Means%2C_DBSCAN%2C_HDBSCAN_(%EA%B5%B0%EC%A7%91%ED%99%94).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 11. K-Means, DBSCAN, HDBSCAN (군집화)

## 개요

**군집화(Clustering)**는 데이터를 **레이블 없이 그룹으로 나누는 비지도 학습(unsupervised learning)** 기법입니다.  
비슷한 특성을 가진 데이터들을 같은 군집(cluster)으로 묶어 패턴이나 구조를 파악합니다.

---

## 1. K-Means 알고리즘

### 원리

K-Means는 다음과 같은 방식으로 동작합니다:

1. $k$개의 중심(centroid)을 무작위로 초기화
2. 각 데이터를 가장 가까운 중심에 할당
3. 각 군집의 중심을 새로 계산
4. 중심이 더 이상 이동하지 않을 때까지 반복

---

### 수식

각 점 $x_i$와 중심 $\mu_j$ 사이의 유클리드 거리로 군집을 정의합니다:

$$
\text{argmin}_{C} \sum_{j=1}^k \sum_{x_i \in C_j} \| x_i - \mu_j \|^2
$$

- $C_j$: 군집 $j$
- $\mu_j$: 군집 $j$의 중심
- $\| x_i - \mu_j \|$: 거리

---

### 특징

- 속도가 빠르고 구현이 쉬움
- 군집 수 $k$를 미리 정해야 함
- 구형(spherical) 군집에 잘 작동
- 이상치(outlier)에 민감함

---

## 2. DBSCAN (Density-Based Spatial Clustering of Applications with Noise)

### 원리

DBSCAN은 밀도 기반 군집화 알고리즘으로, **밀집된 영역은 군집**, **희소한 영역은 이상치**로 간주합니다.

- $\varepsilon$: 반지름
- minPts: 군집을 형성하기 위한 최소 점 수

### 주요 개념

- **Core Point**: 반지름 $\varepsilon$ 안에 minPts 이상 존재
- **Border Point**: Core Point 주변에 있지만 자체는 Core 아님
- **Noise Point**: 어떤 군집에도 속하지 않음

---

### 특징

- 군집 수를 미리 정할 필요 없음
- 이상치 탐지 가능
- 복잡한 형태의 군집도 탐지 가능
- 밀도 설정에 민감

---

## 3. HDBSCAN (Hierarchical DBSCAN)

### 특징

- DBSCAN의 단점을 보완한 **계층적 밀도 기반 군집화**
- $\varepsilon$ 없이도 작동
- 밀도가 다른 군집 탐지 가능
- 자동으로 군집 수 결정

---

## 비교 요약

| 알고리즘 | 군집 수 입력 | 이상치 처리 | 비모양 군집 | 밀도 자동 조절 |
|----------|--------------|-------------|--------------|----------------|
| K-Means  | 필요         | 불가능      | 어려움       | 불가능         |
| DBSCAN   | 불필요       | 가능        | 가능         | 불가능         |
| HDBSCAN  | 불필요       | 가능        | 가능         | 가능           |

---

## 구현 흐름 요약

- K-Means: 평균 중심 갱신
- DBSCAN: 반경 내 밀도 탐색
- HDBSCAN: DBSCAN + 계층적 군집 안정성 기반 선택

---

## 실습 예제 설명

- 2차원 데이터 생성
- NumPy로 K-Means 수동 구현
- Plotly로 시각화
- Scikit-learn, HDBSCAN 라이브러리로 비교


In [1]:
import numpy as np
import plotly.express as px
from sklearn.datasets import make_blobs

# 데이터 생성
X, _ = make_blobs(n_samples=300, centers=3, cluster_std=0.8, random_state=42)

# K-Means 수동 구현
def kmeans_numpy(X, k=3, max_iter=100):
    np.random.seed(42)
    centroids = X[np.random.choice(len(X), k, replace=False)]
    for _ in range(max_iter):
        dists = np.linalg.norm(X[:, np.newaxis] - centroids, axis=2)
        labels = np.argmin(dists, axis=1)
        new_centroids = np.array([X[labels == i].mean(axis=0) for i in range(k)])
        if np.allclose(centroids, new_centroids):
            break
        centroids = new_centroids
    return labels, centroids

labels_km, centers = kmeans_numpy(X, k=3)

# 시각화
px.scatter(x=X[:,0], y=X[:,1], color=labels_km.astype(str),
           title="NumPy K-Means 결과", labels={"x": "X1", "y": "X2"})


In [2]:
from sklearn.cluster import KMeans, DBSCAN
import hdbscan
import pandas as pd

# KMeans
labels_lib_km = KMeans(n_clusters=3, random_state=42).fit_predict(X)

# DBSCAN
labels_db = DBSCAN(eps=0.8, min_samples=5).fit_predict(X)

# HDBSCAN
labels_hdb = hdbscan.HDBSCAN(min_cluster_size=10).fit_predict(X)

# DataFrame
df = pd.DataFrame(X, columns=["x", "y"])
df["KMeans"] = labels_lib_km
df["DBSCAN"] = labels_db
df["HDBSCAN"] = labels_hdb

# 시각화
fig1 = px.scatter(df, x="x", y="y", color=df["KMeans"].astype(str), title="KMeans (sklearn)")
fig2 = px.scatter(df, x="x", y="y", color=df["DBSCAN"].astype(str), title="DBSCAN (sklearn)")
fig3 = px.scatter(df, x="x", y="y", color=df["HDBSCAN"].astype(str), title="HDBSCAN (hdbscan)")
fig1.show()
fig2.show()
fig3.show()


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

